In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv



In [10]:
# Check if GPU is available
is_available = torch.cuda.is_available()

print(f"Is GPU available? {is_available}")

if is_available:
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

Is GPU available? True
GPU Name: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [11]:
#In this dataset, each node is a scientific paper and each edge represents a citation from one paper to another.
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

In [12]:
print(f'(Dataset: {dataset.name})')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Number of features per node: {data.num_features}')
print(f'Number of classes: {dataset.num_classes}')


(Dataset: Cora)
Number of nodes: 2708
Number of edges: 10556
Number of features per node: 1433
Number of classes: 7


In [13]:
# --- 3. Define the Graph Convolutional Network (GCN) Model ---

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [14]:

# --- 4. Train the Model ---

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)

    # Corrected loss calculation with boolean mask
    loss = F.nll_loss(out[data.train_mask.to(torch.bool)], data.y[data.train_mask.to(torch.bool)])

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')


Epoch 000, Loss: 1.9560
Epoch 010, Loss: 0.8027
Epoch 020, Loss: 0.2608
Epoch 030, Loss: 0.1004
Epoch 040, Loss: 0.0598
Epoch 050, Loss: 0.0446
Epoch 060, Loss: 0.0501
Epoch 070, Loss: 0.0437
Epoch 080, Loss: 0.0348
Epoch 090, Loss: 0.0404
Epoch 100, Loss: 0.0618
Epoch 110, Loss: 0.0351
Epoch 120, Loss: 0.0385
Epoch 130, Loss: 0.0350
Epoch 140, Loss: 0.0300
Epoch 150, Loss: 0.0331
Epoch 160, Loss: 0.0283
Epoch 170, Loss: 0.0593
Epoch 180, Loss: 0.0428
Epoch 190, Loss: 0.0263


In [15]:

# --- 5. Evaluate the Model ---

model.eval()
_, pred = model(data).max(dim=1)

# Corrected evaluation with boolean mask
test_mask_bool = data.test_mask.to(torch.bool)
correct = pred[test_mask_bool].eq(data.y[test_mask_bool]).sum().item()
acc = correct / test_mask_bool.sum().item()
print(f'\nAccuracy on test set: {acc:.4f}')



Accuracy on test set: 0.8020


In [16]:

# --- 6. Save the Trained Model ---

torch.save(model.state_dict(), 'gcn_cora.pth')
print('\nModel saved to gcn_cora.pth')


Model saved to gcn_cora.pth
